In [1]:
import pandas as pd
import glob

headers = ['Date','Time','Open','High','Low','Close','Volume']
path = "C:\\Users\\desma\\Downloads\\jupyter work\\DATA\\"
#set up so i could run multiple datasets if wanted
filenames = glob.glob(path + "QQQ15m.txt")
for file in filenames:
    print("\nReading file = ", file)
    df = pd.read_csv(file, header = None)
    df.columns=headers

dfLen = len(df)


Reading file =  C:\Users\desma\Downloads\jupyter work\DATA\QQQ15m.txt


In [3]:
signal = [0]*dfLen
entryBull = [None]*dfLen
entryBear = [None]*dfLen

for row in range(1, dfLen): 
    diffMin = abs(df.Open[row-1]-df.Close[row-1]) * 1.25
    bodyDiff = abs(df.Open[row]-df.Close[row]) 
    #BEARISH reversal: signal = 2
    if(bodyDiff>diffMin and df.Open[row-1]<df.Close[row-1] and df.Open[row]>df.Close[row] 
       and df.Open[row]>df.Close[row-1] and df.Close[row]<df.Open[row-1]):\
        signal[row] = 2 
            
    #BULLISH reversal: signal = 1
    elif(bodyDiff>diffMin and df.Open[row-1]>df.Close[row-1] and df.Open[row]<df.Close[row] 
       and df.Open[row]<df.Close[row-1] and df.Close[row]>df.Open[row-1]):
        signal[row] = 1 

df['Signal'] = signal
df['Signal'].value_counts()



0    21929
2      882
1      863
Name: Signal, dtype: int64

In [5]:
def SIGNAL():
    return df.Signal

In [14]:
from backtesting import Strategy, Backtest
import numpy as np

class myStrat(Strategy):
    def init(self):
        super().init()
        self.signal1=self.I(SIGNAL)     
        
    def next(self):
        super().next()
        trailSL = self.data.Close[-1]*.005
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - trailSL)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + trailSL)
        
        if self.signal1==1 and len(self.trades)==0: #Long Signal
            self.buy()
            
        if self.signal1==2 and len(self.trades)==0: #Short Signal
            self.sell()
            
bt = Backtest(df, myStrat, cash=100_000, margin = 1/1, commission=.000)
stat = bt.run()
stat
#bt.plot(show_legend=False, superimpose = True)

C:\Users\desma\AppData\Local\Temp/ipykernel_18784/1308077041.py:24: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, myStrat, cash=100_000, margin = 1/1, commission=.000)


Start                                     0.0
End                                   23673.0
Duration                              23673.0
Exposure Time [%]                   61.996283
Equity Final [$]                  72728.90745
Equity Peak [$]                   104485.5818
Return [%]                         -27.271093
Buy & Hold Return [%]               -9.083096
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -33.857743
Avg. Drawdown [%]                   -1.669841
Max. Drawdown Duration                23331.0
Avg. Drawdown Duration             983.791667
# Trades                                747.0
Win Rate [%]                        34.672021
Best Trade [%]                       4.026873
Worst Trade [%]                     -1.628918
Avg. Trade [%]                    